<a href="https://colab.research.google.com/github/ludmilaexbrayat/recognizing-packagings/blob/master/Data_Gathering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Google collab setp


In [0]:
!pip install -U -q PyDrive
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

getting the csv 


In [0]:

# csv file from open food fact is available at : https://fr.openfoodfacts.org/data
# strcture of the dataset is detailed here : https://static.openfoodfacts.org/data/data-fields.txt

link_open_food_fact = 'https://drive.google.com/open?id=1vt45SNlhYjQC_uNETUqPwR2jK5o3vW04'

fluff, id_open_food_fact = link_open_food_fact.split('=')

# dictionary below is a correspondance between the main label we want to predict and all the numerous labels from open foodfact
# appearing at least 30 in the dataset. it has been manually labelled by us

link_dico = 'https://drive.google.com/open?id=1QPJWoGtDP6H-4wV3OJHXiDP7r2i6wP_6'

fluff, id_dico = link_dico.split('=')




In [0]:
downloaded = drive.CreateFile({'id':id_open_food_fact}) 
downloaded.GetContentFile('en.openfoodfacts.org.products.csv')

downloaded = drive.CreateFile({'id':id_dico}) 
downloaded.GetContentFile('Voc_dico.xlsx')

In [0]:
#data = pd.read_csv('small_test.csv', delimiter=',', encoding='UTF-8',index_col=0)
data = pd.read_csv('en.openfoodfacts.org.products.csv', delimiter='\t', encoding='UTF-8')
dico = pd.read_excel('Voc_dico.xlsx')

In [5]:
dico.head()

,Label,Col,Col1
0,plastique,Plastique,NaN
1,plastiktute,Plastique,NaN
2,2,NaN,NaN
3,4-pe-ld,NaN,NaN
4,fr-bouteille-verre,Verre,Bouteille


In [6]:
data.head(2)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,...,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,17,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### fonction


In [0]:
import urllib.request
import pickle
import numpy as np

def image_dowloading(data,data_img='/content/data_img/'):
    """
    Downloading of the images based on the openfoodfact csv file
    
    Parameters
    ----------
    data : dataframe pandas
        openfoofact csv file converted into pandas dataframe   
        
    path_img : string
        path of the folder where the images are going to be downloaded
    """ 
    if not os.path.exists(data_img):
        %mkdir data_img
        %cd data_img
    else:
        %cd data_img
        
    
        
    data = data[['code','image_url','packaging_tags']].dropna(subset=['packaging_tags','image_url'])
    data.drop_duplicates(inplace=True,subset=['code'])
    data.reset_index(inplace=True)

    id_downloaded = []
    count_error = 0

    for i in range(0,len(data)):
        try:
            urllib.request.urlretrieve(data['image_url'][i] , str(data['code'][i])+".jpg")
            id_downloaded.append(data['code'][i])
        except Exception as e:
            count_error = count_error + 1
    print(str(count_error)+ ' images couldnt be downloaded')
    print(str(len(id_downloaded))+ ' images has been downloaded')
    np.save('id_downloaded.npy',id_downloaded)
    

In [0]:
import pandas as pd 
def y_target_creation(data,df_dic):
    """
    Creation of y_target dataframe based on the openfoodfact csv file
    
    Parameters
    ----------
    data : dataframe pandas
        openfoofact csv file converted into pandas dataframe   
        
    df_dic : dataframe pandas
        manually completed dictionary of correspondance between major labels and labels in the original
        openfoodfact csv files ( occuring at least 30 times )
    """    
    data = data[~data['image_url'].isna()]
    data= data[['code', 'packaging_tags']].dropna().drop_duplicates('code')
    df_1 = data.set_index('code')['packaging_tags'].str.split(',', expand=True)
    value_vars_1 = df_1.columns 
    df_1 = df_1.reset_index()
    df_1 = pd.melt(df_1, id_vars = 'code', value_vars = value_vars_1).dropna()[['code','value']]
    #df_dic = pd.read_excel('Voc_dico.xlsx')
    dic_transfo1 = dict(zip(df_dic.Label, df_dic.Col))
    dic_transfo2 = dict(zip(df_dic.Label, df_dic.Col1))
    ##apply les dicos 
    df_1['brico 1'] = df_1['value'].map(dic_transfo1)
    df_1['brico 2'] = df_1['value'].map(dic_transfo2)
    df_2 = pd.melt(df_1, id_vars='code', value_vars = ['brico 1', 'brico 2']).dropna()[['code','value']]
    one_hot = pd.get_dummies(df_2['value'])
    y_target = pd.concat([df_2['code'],one_hot],axis=1)
    y_target['code']=y_target['code'].astype(str)
    y_target = y_target.reset_index().groupby('code').sum().drop(columns='index')
    y_target[y_target > 1] = 1
    return y_target
    

In [0]:
import os
import random
import shutil


def img_train_test_split(img_source_dir, train_size):
    """
    Randomly splits images over a train and validation folder, while preserving the folder structure
    
    Parameters
    ----------
    img_source_dir : string
        Path to the folder with the images to be split. Can be absolute or relative path   
        
    train_size : float
        Proportion of the original images that need to be copied in the subdirectory in the train folder
    """    
    if not (isinstance(img_source_dir, str)):
        raise AttributeError('img_source_dir must be a string')
        
    if not os.path.exists(img_source_dir):
        raise OSError('img_source_dir does not exist')
        
    if not (isinstance(train_size, float)):
        raise AttributeError('train_size must be a float')
        
    # Set up empty folder structure if not exists
    if not os.path.exists('/content/data_img'):
      %cd /content/
      %mkdir data_img

    else:
      if not os.path.exists('/content/data_img/train'):
        %cd /content/data_img/
        %mkdir train
      if not os.path.exists('/content/data_img/validation'):
        %cd /content/data_img/
        %mkdir validation
    
    ignored = {"train", "test", "validation","id_downloaded.npy"}
    list_files = [x for x in os.listdir(img_source_dir) if x not in ignored]
    indicies_train = random.sample(range(len(list_files)), int(train_size*len(list_files)))
    train_list = [list_files[i] for i in indicies_train]
    test_list = [x for x in list_files if x not in train_list]

    for i in range(0,len(train_list),1):
      shutil.move("/content/data_img/"+str(train_list[i]),
           "/content/data_img/train/"+str(train_list[i]))
    for i in range(0,len(test_list),1):
      shutil.move("/content/data_img/"+str(test_list[i]),
           "/content/data_img/validation/"+str(test_list[i]))

download images

In [10]:
%cd /content/
image_dowloading(data=data,data_img='/content/data_img/')

/content
/content/data_img
0 images couldnt be downloaded
3 images has been downloaded


y_target creation


In [0]:
y_target = y_target_creation(data=data,df_dic=dico)

train/test split for images


In [12]:
img_train_test_split('/content/data_img',0.7)

/content/data_img
/content/data_img
